In [1]:
import os
import io

import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [2]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [56]:
openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-5-nano"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620" # CLAUDE_MODEL = "claude-3-haiku-20240307"

In [4]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an windows 10. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [27]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. Add a pause in the end of the code so it waits button press after execution. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [7]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [8]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [9]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [10]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [11]:
exec(pi)

Result: 3.141592658589
Execution Time: 9.382045 seconds


In [12]:
optimize_gpt(pi)

#include <iostream>
#include <iomanip>
#include <chrono>

int main() {
    const long long iterations = 100000000LL;
    const double param1 = 4.0;
    const double param2 = 1.0;

    double result = 1.0;

    auto start = std::chrono::high_resolution_clock::now();
    for (long long i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= 1.0 / j;
        j = i * param1 + param2;
        result += 1.0 / j;
    }
    auto end = std::chrono::high_resolution_clock::now();

    double final_result = result * 4.0;

    std::cout.setf(std::ios::fixed);
    std::cout << std::setprecision(12);
    std::cout << "Result: " << final_result << "\n";

    std::chrono::duration<double> elapsed = end - start;
    std::cout << std::setprecision(6);
    std::cout << "Execution Time: " << elapsed.count() << " seconds" << std::endl;

    return 0;
}

In [13]:
exec(pi)

Result: 3.141592658589
Execution Time: 9.346793 seconds


In [24]:
!g++ -O2 -std=c++17 optimized.cpp -o optimized
!./optimized

"." non e riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [30]:
optimize_claude(pi)

```cpp
 <iostream>
omanip>e <i
ono>lude <chr
include <conio.h>

 calculate(long long int iterations, double param1, double param2) {
 result = 1.0;
 long int i = 1; i <= iterations; i++) {
double j = i * param1 - param2;
 (1.0 / j);ult -=
 * param1 + param2;
 (1.0 / j);ult +=
    }
    return result;
}

 main() {
 start_time = std::chrono::high_resolution_clock::now();
 result = calculate(100000000, 4, 1) * 4;
d_time = std::chrono::high_resolution_clock::now();

 << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
cout << "Execution Time: " << std::chrono::duration_cast<std::chrono::duration<double>>(end_time - start_time).count() << " seconds" << std::endl;

d::cout << "Press any key to exit..." << std::endl;
_getch();
;   return 0
}
```

In [31]:
!g++ -O2 -std=c++17 optimized.cpp -o optimized
!./optimized
    

"." non e riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [32]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [33]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 34.608083 seconds


In [57]:
optimize_gpt(python_hard)

#include <iostream>
#include <iomanip>
#include <cstdint>
#include <limits>
#include <chrono>

static inline uint32_t lcg_next(uint32_t value) {
    const uint64_t a = 1664525ULL;
    const uint64_t c = 1013904223ULL;
    const uint64_t m = 0x100000000ULL; // 2^32
    uint64_t t = a * value + c;
    t %= m;
    return static_cast<uint32_t>(t);
}

static inline int64_t max_subarray_sum_with_seed(uint32_t seed, int n, int min_val, int max_val) {
    int range = max_val - min_val + 1;
    int64_t max_ending_here = 0;
    int64_t max_so_far = std::numeric_limits<int64_t>::min();
    uint32_t v = seed;
    bool started = false;
    for (int i = 0; i < n; ++i) {
        v = lcg_next(v);
        int x = static_cast<int>(v % range) + min_val;
        if (!started) {
            max_ending_here = x;
            max_so_far = x;
            started = true;
        } else {
            max_ending_here = (max_ending_here > 0) ? max_ending_here + x : x;
            if (max_ending_here > max_so_far) 

In [58]:
!g++ -O2 -std=c++17 optimized.cpp -o optimized
!./optimized

"." non e riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [47]:
optimize_claude(python_hard)

#include <iostream>
#include <vector>
ono>lude <chr
 <limits>
>include <algorithm
stdint>e <c
ip>clude <ioman
onio.h>e <c

 LCG {
private:
64_t value;
64_t a = 1664525;
_t c = 1013904223;
_t m = 1ULL << 32;

public:
uint64_t seed) : value(seed) {}

64_t next() {
 (a * value + c) % m;
        return value;
    }
};

_t max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
g(seed);lc
<int64_t> random_numbers(n);
 (int i = 0; i < n; ++i) {
[i] = lcg.next() % (max_val - min_val + 1) + min_val;
    }

t max_sum = std::numeric_limits<int64_t>::min();
int64_t current_sum = 0;
 = 0; i < n; ++i) {
_sum = std::max(current_sum + random_numbers[i], random_numbers[i]);
 = std::max(max_sum, current_sum);
}   
;   return max_sum
}

64_t total_max_subarray_sum(int n, uint64_t initial_seed, int min_val, int max_val) {
 total_sum = 0;
CG lcg(initial_seed);
 = 0; i < 20; ++i) {
uint64_t seed = lcg.next();
sum += max_subarray_sum(n, seed, min_val, max_val);
    }
;   return total_sum
}

 main

In [48]:
!g++ -O2 -std=c++17 optimized.cpp -o optimized
!./optimized

"." non e riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [59]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [60]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [61]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far      

In [63]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code: ", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code: ", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [65]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [64]:
def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["g++", "-O2", "-std=c++17", "optimized.cpp", "-o", "optimized"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [66]:
# !g++ -O2 -std=c++17 optimized.cpp -o optimized
# !./optimized

In [67]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [68]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
